<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/DeepLearningNLP/IMDB_report_201931195%EA%B9%80%EC%A0%95%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용할 패키지 로드

In [ ]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization, SpatialDropout1D, Activation
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

# 데이터 로드

In [ ]:
# loss, acc 시각화 함수
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
vocab_size = 10000  # final field
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)
print(X_train.shape)
#0th sentence

print(len(X_train[0]))

#1th sentence
print(len(X_train[1]))

max_len = 200
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

print('X_train의 크기(shape) :',X_train.shape)
print('X_test의 크기(shape) :',X_test.shape)
print(y_train[:5])

In [ ]:
X_train[0]# 맨 마지막 숫자가 32인 경우, 더 긴 문장을 자른 형태

In [ ]:
X_train[1]# 문장의 수가 모자란 경우 앞에서 부터 0으로 채운다.

# Model Build

In [ ]:
inputs = layers.Input(shape=(None,), dtype='int64')

embedded_sequences = layers.Embedding(input_dim = vocab_size, output_dim = 256)(inputs)
x = layers.Dropout(0.3)(embedded_sequences)
x = layers.Conv1D(256, 3, padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(activation='relu')(x)
x = layers.GlobalMaxPool1D()(x)

x = layers.Dense(128)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation(activation='relu')(x)

x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)


model.compile(optimizer=optimizers.Adam(1e-3),
              loss='binary_crossentropy',
              metrics=['acc'])

# 최고 학습 모델 저장하기

In [ ]:
es = EarlyStopping(monitor = 'val_loss',
                   mode = 'min',
                   verbose = 1,
                   patience = 3)
mc = ModelCheckpoint('best_model.h5', 
                     monitor = 'val_acc', 
                     mode = 'max', 
                     verbose = 1, 
                     save_best_only = True)

# 모델학습

In [ ]:
history = model.fit(X_train,
                    y_train,
                    epochs = 3,
                    validation_data = (X_test, y_test),
                    callbacks=[es, mc])

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
plot_graphs(history, 'acc')
plot_graphs(history, 'loss')


# hyper parameter 튜닝하기



In [ ]:
# del model  # 위에서 학습한 모델은 삭제

history_arr = []  # history 저장
eval_arr = []  # evaluate 값 저장

def run_model(dropout_rate, learning_rate):
  inputs = layers.Input(shape=(None,), dtype='int64')

  embedded_sequences = layers.Embedding(input_dim=vocab_size,
                                        output_dim=256)(inputs)

  x = layers.Dropout(dropout_rate)(embedded_sequences)
  x = layers.Conv1D(256, 3, padding='valid')(x)
  x = layers.Activation(activation='relu')(x)
  x = layers.GlobalMaxPool1D()(x)
  x = layers.BatchNormalization()(x)

  x = layers.Dropout(dropout_rate)(x)
  x = layers.Dense(128)(x)
  x = layers.BatchNormalization()(x)
  x = layers.Activation(activation='relu')(x)

  x = layers.Dropout(0.5)(x)

  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = models.Model(inputs, outputs)

  best_model_name = 'best_model_{}_{}.h5'.format(str(learning_rate)[2:], round(dropout_rate * 10))

  es = EarlyStopping(monitor='val_loss', 
                   mode='min', 
                   verbose=2, 
                   patience=2)

  mc = ModelCheckpoint(best_model_name, 
                      monitor='val_acc', 
                      mode='max', 
                      verbose=2, 
                      save_best_only=True)

  model.compile(optimizer=optimizers.Adam(learning_rate),
                loss='binary_crossentropy',
                metrics=['acc'])
  
  history = model.fit(X_train,
                    y_train,
                    epochs=2,
                    verbose=2,
                    validation_data=(X_test, y_test),
                    callbacks=[es, mc])
  
  loaded_model = load_model(best_model_name)
  print("\n [%s], 테스트 정확도: %.4f" % (best_model_name, loaded_model.evaluate(X_test, y_test)[1]))
  eval_arr.append(loaded_model.evaluate(X_test, y_test)[1])
  history_arr.append(history)
  return model

In [ ]:
import numpy as np
dropout_arr = np.arange(0.1, 1.0, 0.1)  # 최적화 dropout 찾기

for x in [1e-2, 1e-3, 1e-4]:
  print("=" * 33)
  print(" start learning_rate :: {}".format(x))
  for j in dropout_arr:
    print("--" * 20)
    print(" dropout rate : {}".format(j))
    print("--" * 20)
    rate = np.round(j, 1)
    run_model(rate, x)
  print("=" * 33)

# accuracy 및 loss 시각화

In [ ]:
for i, x in enumerate(history_arr):
  print("==" * 30)
  print("history idx : %d" % i)
  plot_graphs(x, 'acc')
  plot_graphs(x, 'loss')
  print("==" * 30)